In [16]:
import pandas as pd
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt

First, we have designed a very simple 5x5 treasure map with 6 obstacles and 1 player where that player has to move through the map and avoid the obstacles in order to get to the treasure. We also use the Q-learning algorithm for the user to pick the best moves through the map.

Roughly the map, player and obstacles look something like as below where:
P : Player
O : Obstacle
E: Empty
T: Treasure
O1: Different kind of obstacle

P O O E E
E O E E E
E E E O E
E O E O E
E O1 E E T

The player starts at 0 and can move up/down/left/right or stay in the same state without any movement. Each move that the player chooses will take them to a cell in the gridand the goal is for the player to reach the treasure at cell 24 assuming we start the count from 0. The map has a number of obstacles interspersed throughout. 

We'll start by setting up the reward system and giving the agent some rewards for its action. The rewards will have values respectively for each of the obstacles O and O1, the treasure, a valid / invalid move are: -5, -10, 10, 1 and 0. We'll consider the route with maximized score/points.  

In [2]:
reward = np.array([[0, 1, 0, -5, 1],
                   [0, -5, 1, -5, -5],
                   [0, 1, 5, 1, -5],
                   [0, 1, -5, 1, 1],
                   [0, 1, 1, 0, 1],
                   [1, 1, 0, -5, 1],
                   [-5, 1, 1, 1, -5],
                   [-5, 1, -5, 1, 1],
                   [1, -5, 1, 1, 1],
                   [1, 1, 1, 0, 1],
                   [1, 1, 0, 1, 1],
                   [-5, -5, 1, 1, 1],
                   [1, 1, 1, -5, 1],
                   [1, -5, 1, 1, -5],
                   [1, 1, -5, 0, 1],
                   [1, 1, 0, -5, 1],
                   [1, -10, 1, 1, -5],
                   [1, 1, -5, -5, 1],
                   [-5, 1, 1, 1, -5],
                   [1, 10, -5, 0, 1],
                   [1, 0, 0, -10, 1],
                   [-5, 0, 1, 1, -10],
                   [1, 0, -10, 1, 1],
                   [-5, 0, 1, 10, 1],
                   [1, 0, 1, 0, 10]
                  ])

We use Q-learning which attempts to learn the value in a given state and take a specific action there. This table is gradually updated as we observe the rewards the agent obtain for various actions using Bellman Equation: Q(s,a) = r + γ(max(Q(ś,á))

s : current state
a : action from current state
ś : state from action
r : reward for action
γ : discount factor

In [11]:
learning_rate = 0.7
q_grid = np.zeros((25,5))

The transition grid below displays all the possible moves to another state from a given state. An invalid move is represented by -1 

In [6]:
transition_grid = np.array([[-1, 5, -1, 1, 0],
                       [-1, 6, 0, 2, 1],
                       [-1, 7, 1, 3, 2],
                       [-1, 8, 2, 4, 3],
                       [-1, 9, 3, -1, 4],
                       [0, 10, -1, 6, 5],
                       [1, 11, 5, 7, 6],
                       [2, 12, 6, 8, 7],
                       [3, 13, 7, 9, 8],
                       [4, 14, 8, -1, 9],
                       [5, 15, -1, 11, 10],
                       [6, 16, 10, 12, 11],
                       [7, 17, 11, 13, 12],
                       [8, 18, 12, 14, 13],
                       [9, 19, 13, -1, 14],
                       [10, 20, -1, 16, 15],
                       [11, 21, 15, 17, 16],
                       [12, 22, 16, 18, 17],
                       [13, 23, 17, 19, 18],
                       [14, 24, 18, -1, 19],
                       [15, -1, -1, 21, 20],
                       [16, -1, 20, 22, 21],
                       [17, -1, 21, 23, 22],
                       [18, -1, 22, 24, 23],
                       [19, -1, 23, -1, 24]
                        ])

Given a particular state an action grid allows the agent moves in a certian direction: up, down, left, right are represnted as 0,1,2,3 and 4

In [8]:
action_grid = np.array([[1, 3, 4],
                   [1, 2, 3, 4],
                   [1, 2, 3, 4],
                   [1, 2, 3, 4],
                   [1, 2, 4],
                   [0, 1, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 4],
                   [0, 1, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 4],
                   [0, 1, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 3, 4],
                   [0, 1, 2, 4],
                   [0, 3, 4],
                   [0, 2, 3, 4],
                   [0, 2, 3, 4],
                   [0, 2, 3, 4],
                   [0, 2, 4]])

In [12]:
for i in range(500):
    #Start with the first most state
    start = 0
    current = start
    
    #Keep moving forward until the goal state is reached
    while current != 24:
        #Select one among all possible actions for the current state
        action = random.choice(action_grid[current])
        
        #Travel to the next state as a result of the action taken previously
        next_state = transition_grid[current][action]
        future_reward = []
        
        #Add all the current rewards value for all possible actions
        for action_next in action_grid[next_state]:
            future_reward.append(q_grid[next_state][action_next])
        
        #For all possible actions from the next state, select the one with highest Q value
        q_state = reward[current][action] + learning_rate*max(future_reward)
        
        #Update the Q table with new reward value
        q_grid[current][action] = q_state
        
        #Set the next state as the current state in order to move forward
        current = next_state

In [13]:
q_grid

array([[ 0.        ,  3.882362  ,  0.        , -2.39764262,  3.7176534 ],
       [ 0.        , -1.88234   ,  3.7176534 ,  0.32165017, -2.39764262],
       [ 0.        ,  4.4538    ,  7.60235738,  4.4538    ,  0.32165017],
       [ 0.        ,  4.934     ,  0.32165017,  4.934     ,  4.4538    ],
       [ 0.        ,  5.62      ,  4.4538    ,  0.        ,  4.934     ],
       [ 3.7176534 ,  4.11766   ,  0.        , -1.88234   ,  3.882362  ],
       [-2.39764262,  4.4538    ,  3.882362  ,  4.4538    , -1.88234   ],
       [ 0.32165017,  4.934     , -1.88234   ,  4.934     ,  4.4538    ],
       [ 4.4538    , -0.38      ,  4.4538    ,  5.62      ,  4.934     ],
       [ 4.934     ,  6.6       ,  4.934     ,  0.        ,  5.62      ],
       [ 3.882362  ,  3.882362  ,  0.        ,  4.4538    ,  4.11766   ],
       [-1.88234   , -1.066     ,  4.11766   ,  4.934     ,  4.4538    ],
       [ 4.4538    ,  5.62      ,  4.4538    , -0.38      ,  4.934     ],
       [ 4.934     ,  0.6       ,  4.9

In [17]:
#convert ot dataframe
df = pd.DataFrame(q_grid)
df

,0,1,2,3,4
0,0.000000,3.882362,0.000000,-2.397643,3.717653
1,0.000000,-1.882340,3.717653,0.321650,-2.397643
2,0.000000,4.453800,7.602357,4.453800,0.321650
3,0.000000,4.934000,0.321650,4.934000,4.453800
4,0.000000,5.620000,4.453800,0.000000,4.934000
5,3.717653,4.117660,0.000000,-1.882340,3.882362
6,-2.397643,4.453800,3.882362,4.453800,-1.882340
7,0.321650,4.934000,-1.882340,4.934000,4.453800
8,4.453800,-0.380000,4.453800,5.620000,4.934000
9,4.934000,6.600000,4.934000,0.000000,5.620000


In [20]:
route = []
state = 0
while state != 24:
    route.append(state)
    row = df.iloc[state]
    direction = row.idxmax(axis=1)
    state = transition_grid[state][direction]
route.append(state)
route

[0, 5, 10, 11, 12, 17, 22, 23, 24]

The route printed above is the optimal route to be taken by the player in order to get to the treasure

When capturing the above route in the sample map that wwe had above we find that the route the player is to take to get to the treasure is :

P O O E E
| O E E E
|____ O E
    |O E
     |__T


This way the player will reach the treasure using Q-learning. 

